# association
co_visitaion_matrixにaidごとの出現数を考慮する

In [6]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf

In [7]:
SEED = 42
random.seed(SEED)

In [8]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [9]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")
train_sessions = train_sessions.drop(columns=["type", "ts"])
test_sessions = test_sessions.drop(columns=["type", "ts"])
sessions = pd.concat([train_sessions, test_sessions])
del train_sessions, test_sessions
gc.collect()

pair_df = pd.read_pickle(PREP_DIR + "co_visitation_matrix.pkl")
pair_df.columns = ["aid_x", "aid_y", "pair_cnt"]

In [10]:
aid_cnt = sessions.groupby("aid").count().reset_index()
aid_cnt.columns = ["aid", "single_cnt"]

total_cnt = len(sessions)

In [11]:
pair_df = pair_df.merge(aid_cnt, left_on="aid_x", right_on="aid", how="left").drop(columns=["aid"])
pair_df = pair_df.merge(aid_cnt, left_on="aid_y", right_on="aid", how="left").drop(columns=["aid"])

In [12]:
pair_df["x_support"] = pair_df["single_cnt_x"] / total_cnt
pair_df["y_support"] = pair_df["single_cnt_y"] / total_cnt
pair_df["xy_support"] = pair_df["pair_cnt"] / total_cnt
pair_df["lift"] = pair_df["xy_support"] / (pair_df["x_support"] * pair_df["y_support"])

In [13]:
pair_df[["aid_x", "aid_y", "lift"]].to_pickle(PREP_DIR + f"association.pkl")